<h2> PROYECTO - Aprendizaje No Supervisado<h2>

<h2>Integrantes:<h2>
<h3>- Maximiliano Zapater Cornejo<h3>
<h3>- Javiera Vukasovic Figueroa<h3>
<h3>- Sebastián Silva Espinoza<h3>
<h3>- Glen Restrepo Arteaga<h3>
<h3>- Marco Gutiérrez Corvalán<h3>

Utilizaremos un dataset para estimación de los niveles de obesidad según los hábitos alimenticios y condiciones físicas en 2111 personas de Colombia, Perú y México. 

**El dataset contiene los siguientes atributos**:

* Gender (*binario*): Género [*Female/Male*]

* Age (*numérico continuo*): Edad 
* Height (*numérico continuo*): Altura [*metros*]
* Weight (*numérico continuo)*: Peso [*kilogramos*]
* Family history with overweight (*binario*): ¿Algún miembro de la familia ha sufrido o sufre de sobrepeso?
* FAVC (*binario*): ¿Comes alimentos altos en calorías frecuentemente?
* FCVC (*numérico entero*): ¿Usualmente comes vegetales en tus comidas principales? [*Never/Sometimes/Always*]
* NCP (*numérico continuo*): ¿Cuántas comidas principales consumes diariamente? [*Between 1 and 2/Three/More than three*]
* CAEC (*categórica*): ¿Comes algún alimento entre comidas principales? [*No/Sometimes/Frequently/Always*]
* SMOKE (*binario*): ¿Fumas?
* CH2O (*numérico continuo*): ¿Cuánta agua bebes diariamente? [*Less than a liter/Between 1 and 2L/More than 2L*]
* SCC (*binario*): ¿Monitoreas las calorías que consumes diariamente?
* FAF (*numérico continuo*): ¿Qué tan seguido realizas actividad física? [*I do not have/1 or 2 days/2 or 4 days/4 or 5 days*]
* TUE (*numérico entero*): ¿Cuánto tiempo usas aparatos electrónicos tales como celular, videojuegos, televisión, computadora u otros?
* CALC (*categórica*): ¿Qué tan seguido bebes alcohol? [*I do not drink/Sometimes/Frequently/Always*]
* MTRANS (*categórica*): ¿Qué medio de transporte sueles utilizar? [*Automobile/Motorbike/Bike/Public Transportation/Walking*]

<h3> Análisis Estadístico <h3>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_obesity = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df_obesity.drop(columns = ['NObeyesdad'], inplace = True)
display(df_obesity.head())
display(df_obesity.describe())

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation


,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [17]:
embedding = pd.get_dummies(df_obesity[['family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']])
embedding

,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,...,SCC_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,False,True,True,False,False,False,True,False,True,False,...,False,False,False,False,True,False,False,False,True,False
1,False,True,True,False,False,False,True,False,False,True,...,True,False,False,True,False,False,False,False,True,False
2,False,True,True,False,False,False,True,False,True,False,...,False,False,True,False,False,False,False,False,True,False
3,True,False,True,False,False,False,True,False,True,False,...,False,False,True,False,False,False,False,False,False,True
4,True,False,True,False,False,False,True,False,True,False,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,False,True,False,True,False,False,True,False,True,False,...,False,False,False,True,False,False,False,False,True,False
2107,False,True,False,True,False,False,True,False,True,False,...,False,False,False,True,False,False,False,False,True,False
2108,False,True,False,True,False,False,True,False,True,False,...,False,False,False,True,False,False,False,False,True,False
2109,False,True,False,True,False,False,True,False,True,False,...,False,False,False,True,False,False,False,False,True,False
